In [1]:
import numpy as np
import numpy.linalg as lia
import pandas as pd
import matplotlib as plt

In [2]:
from sklearn import datasets
digits = datasets.load_digits()

In [3]:
print(len(digits.data))
print(digits.target.size)

1797
1797


In [4]:
from sklearn.datasets import fetch_openml
wine = fetch_openml(name='wine', version=1)

In [5]:
print(len(wine.data))
print(wine.target.size)

178
178


In [6]:
# normalization of wine data via division of each feature by its max value

wine_data_norm = []
for col in wine.data.T:
    col_norm = col/np.amax(col)
    wine_data_norm.append(col_norm)
    
wine.data = np.asarray(wine_data_norm).T

In [7]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index+(foldIndex*digitsValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []
    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index+(foldIndex*wineValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []
    
    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)

In [8]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [9]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [10]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=0.01, momentum=0.9, max_iters=20, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.deltas = [] # delta w{t}
        
    def run(self, gradient_fn, x, y, w):
        # TODO: implement correct termination condition
        
        grad = np.inf
        t = 1
        
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            print(f'Grad descent step {t}')            
            gradients = gradient_fn(x, y, w, self.batch_size)
            for c in range(len(y[0])):
                wc = []
                if(c==0):
                    w[c] = w[c-1] - self.learning_rate * gradients[c]
                else:
                    delta_w = (self.momentum)*(self.deltas[-1]) + (1-self.momentum)*gradients[c]
                    w[c] = w[c-1] - self.learning_rate * delta_w
                if(c != 0):
                    w_prev = self.deltas[-1]
                self.deltas.append(w[c])
                print(f'=============\n{c} : {w[c]}\n')
            t += 1
            
        return w


In [11]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        # TODO: add bias
        
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    
                    # TODO: may change, see slide 27 of logistic slideshow
                    cost_c = np.dot(yh_c - y_c, a)
                    
                    gradients[c] += cost_c

            return gradients
        
        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
        return self
    
    def predict(self, x):
        a = np.asarray(x)
        b = np.asarray(self.w)
        
#         if self.add_bias:
#             x = np.column_stack([x,np.ones(N)])

        yh=[]
        for i,x_c in enumerate(a):
            yh_x=[]
            for c in range(len(b)):
                w_x =  b[c] @ x_c
                num = np.exp(w_x)

                den = 0
                for i in range(len(b)):
                    w_x =  b[i] @ x_c
                    den += np.exp(w_x)

                yh_c = num/den
                yh_x.append(yh_c)
            yh.append(yh_x)
        return yh

In [12]:
def accurate(a, b):
    return np.argmax(a) == np.argmax(b)

def cost(yh, y):
    return y * np.log1p(np.exp(-yh)) + (1-yh) * np.log1p(np.exp(yh))

# TODO: grid-search to find lowest cost combination of model hyper-parameters

batch_size = 10
learning_rate = 0.01
momentum = 0.9

digits_accuracy = 0
digits_cost = 0

for fold_index, fold in enumerate(xDigitsTrainingSets):
    gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
    logisticRegressionModel = LogisticRegression(False)
    
    logisticRegressionModel.fit(fold, yDigitsTrainingSets[fold_index], gradientDescentModel)
    yh = logisticRegressionModel.predict(xDigitsValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh):
        if accurate(yh_x, yDigitsValidationSets[fold_index][sample_index]):
            digits_accuracy += 1
        c = np.argmax(yDigitsValidationSets[fold_index][sample_index])
        cst = cost(yh_x[c], yDigitsValidationSets[fold_index][sample_index][c])
        digits_cost += cst
        print(yh_x, yDigitsValidationSets[fold_index][sample_index])
        print(cst)
        
digits_accuracy /= len(digits.data)

wine_accuracy = 0
wine_cost = 0

for fold_index, fold in enumerate(xWineTrainingSets):
    gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
    logisticRegressionModel = LogisticRegression(False)
    
    logisticRegressionModel.fit(fold, yWineTrainingSets[fold_index], gradientDescentModel)
    yh = logisticRegressionModel.predict(xWineValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh):
        if accurate(yh_x, yWineValidationSets[fold_index][sample_index]):
            wine_accuracy += 1
        c = np.argmax(yWineValidationSets[fold_index][sample_index])
        cst = cost(yh_x[c], yWineValidationSets[fold_index][sample_index][c])
        wine_cost += cst
        print(yh_x, yWineValidationSets[fold_index][sample_index])
        print(cst)

wine_accuracy /= len(wine.data)
        
print("Model hyper-parameters:")
print("\tMini-batch size:", batch_size)
print("\tLearning rate:", learning_rate)
print("\tMomentum:", momentum)
print("Digits accuracy:", digits_accuracy)
print("Digits total cost:", digits_cost)
print("Wine accuracy:", wine_accuracy)
print("Wine total cost:", wine_cost)


Grad descent step 1
0 : [ 0.   -0.07 -0.55 -1.27 -1.08 -0.51 -0.12  0.    0.   -0.11 -0.8  -1.25
 -1.02 -0.59 -0.05  0.    0.   -0.1  -0.87 -0.91 -0.68 -0.8  -0.17  0.
  0.   -0.24 -1.13 -1.28 -1.07 -0.89 -0.38  0.    0.   -0.25 -0.74 -1.1
 -1.03 -0.83 -0.31  0.    0.   -0.12 -0.57 -1.05 -0.86 -0.53 -0.4   0.
  0.   -0.09 -0.8  -1.11 -0.78 -0.72 -0.44 -0.01  0.   -0.05 -0.61 -1.04
 -0.98 -0.79 -0.36 -0.12]

1 : [ 0.000e+00 -7.070e-02 -4.255e-01 -9.927e-01 -8.508e-01 -3.251e-01
 -1.112e-01  0.000e+00  0.000e+00 -1.111e-01 -6.580e-01 -8.825e-01
 -6.902e-01 -3.459e-01 -3.050e-02  0.000e+00  0.000e+00 -7.100e-02
 -6.787e-01 -4.391e-01 -3.368e-01 -5.580e-01 -1.717e-01  0.000e+00
  0.000e+00 -1.524e-01 -9.213e-01 -8.628e-01 -7.007e-01 -6.689e-01
 -3.838e-01  0.000e+00  0.000e+00 -2.425e-01 -6.374e-01 -8.510e-01
 -6.203e-01 -6.283e-01 -3.131e-01  0.000e+00  0.000e+00 -1.212e-01
 -4.757e-01 -8.705e-01 -4.186e-01 -3.653e-01 -4.040e-01  0.000e+00
  0.000e+00 -9.090e-02 -6.480e-01 -8.611e-01 -3.8

<ipython-input-11-f50da25bbd75>:29: RuntimeWarning: invalid value encountered in double_scalars
  yh_c = num/den



1 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

2 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

3 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

4 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na


4 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

5 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

6 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

7 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, 

...


Grad descent step 7
0 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

1 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

2 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

3 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan na


5 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

6 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

7 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]

8 : [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, 


1 : [-0.22959562 -0.18765377 -0.19159632 -0.30245904 -0.17401104 -0.046404
  0.04276974 -0.28434407 -0.1032341  -0.05709392 -0.12166935 -0.09380234
  0.09694157]

2 : [-0.22531836 -0.17569866 -0.18659091 -0.28453106 -0.16718601 -0.05430413
  0.02720407 -0.27307049 -0.10471313 -0.05364671 -0.1254079  -0.10118379
  0.07974808]

Grad descent step 14
0 : [-0.30333461 -0.2260781  -0.23301645 -0.35715014 -0.21420577 -0.08818663
  0.02255848 -0.33424948 -0.14752045 -0.05764218 -0.15947994 -0.131268
  0.09888216]

1 : [-0.2647906  -0.20185648 -0.20458515 -0.31508751 -0.19007518 -0.0691753
  0.02909297 -0.29806344 -0.12380652 -0.0521628  -0.13498373 -0.10571791
  0.08937903]

2 : [-0.24440172 -0.18252604 -0.18898975 -0.28938685 -0.17311851 -0.07063581
  0.01837441 -0.27122342 -0.11882029 -0.04736018 -0.129571   -0.10693085
  0.08002525]

Grad descent step 15
0 : [-0.39338368 -0.25348357 -0.3231599  -0.42056419 -0.27888461 -0.14569408
 -0.01368469 -0.37776611 -0.18879572 -0.10724107 -0.21953108


1 : [0.22139932 0.08925299 0.1641276  0.0923086  0.15589816 0.19527516
 0.16367933 0.06947503 0.11615424 0.14513805 0.1437159  0.17482203
 0.23001575]

2 : [0.17571226 0.06883774 0.12695385 0.06452338 0.12351869 0.15818171
 0.13411623 0.04753942 0.09187802 0.12269449 0.11205346 0.13675421
 0.19471954]

Grad descent step 3
0 : [0.39410013 0.11757622 0.30320552 0.14902353 0.29617162 0.372426
 0.31631772 0.09749293 0.24750173 0.24536127 0.24826091 0.34603874
 0.38063593]

1 : [0.35291532 0.10439426 0.27106053 0.13704548 0.26467892 0.33125859
 0.2821984  0.08916317 0.22102335 0.21800913 0.22321961 0.30929087
 0.33925361]

2 : [0.30502996 0.09272503 0.23389641 0.1148273  0.22842978 0.28766996
 0.24423112 0.07569853 0.1897721  0.19139125 0.19220692 0.26613062
 0.29725424]

Grad descent step 4
0 : [0.4095574  0.13738507 0.30919059 0.12624999 0.29735429 0.40719924
 0.34530508 0.08302033 0.25662482 0.23411141 0.2586467  0.40648818
 0.42540619]

1 : [0.36785234 0.12147737 0.27777417 0.11421267 

In [ ]:
# TODO: comparison against another classifier (e.g. KNN)